In [1]:
%matplotlib inline
from IPython.display import Image
import numpy.matlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats
import openpyxl
from scipy.optimize import curve_fit
from mpl_toolkits.mplot3d import Axes3D

import xml.etree.ElementTree as ET
from subprocess import call
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from scipy.interpolate import griddata

import openmc

import Define_Nektar
import Define_OpenMC

import os
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Parameters of reactor
# Unit: cm
# To be discussed: Parameters for reactor to become critical
parameters_dic = {}

parameters_dic.update(fuel_r = 11/2)
parameters_dic.update(fuel_h = 19.5)

parameters_dic.update(controlRod_r = 4.4/2)
parameters_dic.update(controlRod_h_max = 27)
parameters_dic.update(controlRod_l = 24.5)

parameters_dic.update(reflector_r = 42/2)

parameters_dic.update(heat_pipe_R = 1.27/2)
parameters_dic.update(heat_pipe_r = 1.27/2-0.1)

parameters_dic.update(top_distance = 10.5)
parameters_dic.update(bottom_distance = 5)

parameters_dic.update(heat_power = 4000)

parameters_dic.update(reflector_h = parameters_dic['top_distance']+parameters_dic['bottom_distance']+parameters_dic['fuel_h'])

temp_pipe = 1073.5
# Insert control rod, maximum is 27
controlRod_deep = 0
# Number of cells
cells_num_dic = {'n_r':10,'n_r_outer':5,'n_h':20}
# Settings of OpenMC
settings_dic = {'batches':80,'inactive':30,'particles':200000}
#Settings of Nektar++
file_name = 'HeatPipeReactor_3' # fiel_name.xml with settings of Poisson solver
solver_name = 'ADRSolver' 
# Number of iteration
iteration = 10
# Initial temperature distribution in cells
Initial_temperature = 1073.5
temp_cells_mat = Initial_temperature*np.ones((cells_num_dic['n_h'],(cells_num_dic['n_r_outer']+cells_num_dic['n_r'])))
# Get some data
volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep)

In [3]:
x,y,z = Define_Nektar.readNodesFromVtu(file_name)
nodes_dic = {'x':x,'y':y,'z':z}
# Initial temperature distribution in nodes
temp_nodes_vec = Initial_temperature*np.ones(len(x))
fuel_nodes_index = Define_OpenMC.getFuelNodesIndex(nodes_dic,parameters_dic)
# For Test
temp_error_vec = np.zeros(iteration)
k_eff_mean_vec = np.zeros(iteration)
k_eff_dev_vec = np.zeros(iteration)

heat_ratio = np.zeros(iteration)
flux_ratio = np.zeros(iteration)

num_col = cells_num_dic['n_h']*(cells_num_dic['n_r']+cells_num_dic['n_r_outer'])

heat_mean_mat = np.zeros((iteration,num_col))
heat_dev_mat = np.zeros((iteration,num_col))
flux_mean_mat = np.zeros((iteration,num_col))
flux_dev_mat = np.zeros((iteration,num_col))

In [4]:
for i in range(iteration):
    start_tot = time.time()
    # For Test
    temp_cells_mat_last = temp_cells_mat
    
    # Iteration
    print('Iteration: '+str(i+1)+' begins')
    # Run OpenMC !
    time_start = time.time()
    openmc.run(output=False)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC run time:'+str(time_end-time_start))
    
    # Post-process the result of heat source and generate Force.pts
    time_start = time.time()
    k_eff,tally_dic = Define_OpenMC.postProcess(nodes_dic,volume_mat,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic,settings_dic,fuel_cell_ID_list)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' OpenMC post-process time:'+str(time_end-time_start))
    
    # k-eff: mean value and standard deviation
    k_eff_mean_vec[i] = k_eff.nominal_value
    k_eff_dev_vec[i] = k_eff.std_dev
    
    # Heat and flux
    heat_mean_vec = tally_dic['heat_mean']
    heat_dev_vec = tally_dic['heat_dev']
    flux_mean_vec = tally_dic['flux_mean']
    flux_dev_vec = tally_dic['flux_dev']
    
    heat_mean_mat[i,:] = tally_dic['heat_mean']
    heat_dev_mat[i,:] = tally_dic['heat_dev']
    flux_mean_mat[i,:] = tally_dic['flux_mean']
    flux_dev_mat[i,:] = tally_dic['flux_dev']
    
    if i>0:
        heat_index = np.where((heat_mean_mat[i,:]<heat_mean_mat[i-1,:]+heat_dev_mat[i-1,:]) & (heat_mean_mat[i,:]>heat_mean_mat[i-1,:]-heat_dev_mat[i-1,:]))
        flux_index = np.where((flux_mean_mat[i,:]<flux_mean_mat[i-1,:]+flux_dev_mat[i-1,:]) & (flux_mean_mat[i,:]>flux_mean_mat[i-1,:]-flux_dev_mat[i-1,:]))
        heat_ratio[i] = len(heat_index[0])/num_col
        flux_ratio[i] = len(flux_index[0])/num_col
    
    
    # Run Nektar++
    time_start = time.time()
    file_name_new = Define_Nektar.runNektar_Temp(file_name,solver_name,temp_pipe,i)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar run time:'+str(time_end-time_start))
        
    # Post-process result of temperature
    time_start = time.time()
    temp_nodes_vec = Define_Nektar.postProcess_Temp(file_name_new)
    time_end = time.time()
    print('Iteration: '+str(i+1)+' Nektar post-process time:'+str(time_end-time_start))
    temp_cells_mat = Define_OpenMC.getCellTemperature(nodes_dic,temp_nodes_vec,fuel_nodes_index,parameters_dic,cells_num_dic)

    volume_mat,fuel_cell_ID_list = Define_OpenMC.define_Geo_Mat_Set(cells_num_dic,parameters_dic,settings_dic,temp_cells_mat,controlRod_deep)
    # For Test
    
    # Largest relative error of fuel temperature
    temp_error = np.abs(temp_cells_mat-temp_cells_mat_last)/temp_cells_mat_last
    temp_error_vec[i] = temp_error.max()
    
    end_tot = time.time()
    print('Total time: '+str(end_tot-start_tot))
    
#     if (heat_ratio[i]>=0.95) | (flux_ratio[i]>=0.95):
#         break

Iteration: 1 begins
Iteration: 1 OpenMC run time:4384.41474032402
Iteration: 1 OpenMC post-process time:22.001779556274414
Iteration: 1 Nektar run time:208.34228110313416
Iteration: 1 Nektar post-process time:1.9865317344665527
Total time: 4620.640439271927
Iteration: 2 begins
Iteration: 2 OpenMC run time:4473.651674032211
Iteration: 2 OpenMC post-process time:22.58476448059082
Iteration: 2 Nektar run time:227.85153651237488
Iteration: 2 Nektar post-process time:2.2433407306671143
Total time: 4730.3211925029755
Iteration: 3 begins
Iteration: 3 OpenMC run time:4670.902345895767
Iteration: 3 OpenMC post-process time:24.05918312072754
Iteration: 3 Nektar run time:218.01923537254333
Iteration: 3 Nektar post-process time:2.018817901611328
Total time: 4919.179135799408
